# Unit 3

## Handling Empty Files with Grace

You've just learned how to extract file content for AI code review! Now, let's improve our context generator by fixing a bug that appears when handling empty files.

Currently, our get_file_context function tries to split the content into lines without first checking whether that content exists. When a file is empty, this causes the function to process an empty string, which can lead to errors in real-world scenarios.

To fix this issue:

Run the tests to see the bug in action
Add a check that detects when file_record.content is empty
Return an empty string immediately when empty content is detected
This small improvement makes your context generator much more robust. In real code review systems, you'll often encounter empty files (such as newly created files or configuration templates), and your code needs to handle these cases smoothly without crashing.

```python
from typing import List, Dict
from mock_data import Session, CodeFile

def get_file_context(session: Session, file_path: str, max_lines=50) -> str:
    """Get current file content as context"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return ""
    
    # TODO: Add a check for empty content and return an empty string if the content is empty
    
    lines = file_record.content.split('\n')
    if len(lines) <= max_lines:
        return file_record.content
    
    # Return first part of file with truncation notice
    truncated = '\n'.join(lines[:max_lines])
    return f"{truncated}\n... [truncated after {max_lines} lines]"

def get_recent_changes(session: Session, file_path: str, limit=3) -> List[Dict]:
    """Get recent changes to the file"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return []
    
    # This is a stub for demonstration; in a real scenario, this would query commits
    # Here we return a mock list
    return [
        {
            'hash': 'abc12345',
            'message': 'Initial commit',
            'author': 'Alice',
            'date': '2024-06-01'
        },
        {
            'hash': 'def67890',
            'message': 'Refactor code',
            'author': 'Bob',
            'date': '2024-06-02'
        }
    ][:limit]

def find_related_files(session: Session, file_path: str) -> List[str]:
    """Find files that might be related based on imports"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return []
    
    # Simple heuristic: find files imported in this file
    related = []
    content = file_record.content
    
    # Python imports
    for line in content.split('\n'):
        if line.strip().startswith('from ') or line.strip().startswith('import '):
            # Extract module name and try to find corresponding file
            # This is a simplified approach
            if 'import' in line and '.' in line:
                module = line.split()[-1].replace('.', '/') + '.py'
                if session.query(CodeFile).filter_by(file_path=module).first():
                    related.append(module)
    
    return related[:3]  # Limit to 3 related files



## Handling Empty Files in Context Generation

Now that you've learned about extracting file content for AI code review, let's make our context generator more robust! In this exercise, you'll fix a bug in the get_file_context function that causes it to crash when processing files with empty content.

The function currently tries to split an empty string without checking whether the content exists first. This is a common issue when working with real-world files — not all files contain content, and our code needs to handle these edge cases gracefully.

Run the tests to see the bug in action, then modify the function to properly handle empty content by:

Adding a check for empty content before trying to process it
Returning an empty string when the content is empty
Making your context generator handle all types of files, including empty ones, is an important step toward building a reliable AI code review system that won't break in production.

```python
from mock_data import Session, CodeFile
from typing import List, Dict


def get_file_context(session: Session, file_path: str, max_lines=50) -> str:
    """Get current file content as context"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return ""
    
    lines = file_record.content.split('\n')
    if len(lines) <= max_lines:
        return file_record.content
    
    # Return first part of file with truncation notice
    truncated = '\n'.join(lines[:max_lines])
    return f"{truncated}\n... [truncated after {max_lines} lines]"


def get_recent_changes(session: Session, file_path: str, limit=3) -> List[Dict]:
    """Get recent changes to the file"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return []
    
    # This is a stub for demonstration; in a real scenario, this would query commits
    # Here we return a mock list
    return [
        {
            'hash': 'abc12345',
            'message': 'Initial commit',
            'author': 'Alice',
            'date': '2024-06-01'
        },
        {
            'hash': 'def67890',
            'message': 'Refactor code',
            'author': 'Bob',
            'date': '2024-06-02'
        }
    ][:limit]


def find_related_files(session: Session, file_path: str) -> List[str]:
    """Find files that might be related based on imports"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return []
    
    # Simple heuristic: find files imported in this file
    related = []
    content = file_record.content
    
    # Python imports
    for line in content.split('\n'):
        if line.strip().startswith('from ') or line.strip().startswith('import '):
            # Extract module name and try to find corresponding file
            # This is a simplified approach
            if 'import' in line and '.' in line:
                module = line.split()[-1].replace('.', '/') + '.py'
                if session.query(CodeFile).filter_by(file_path=module).first():
                    related.append(module)
    
    return related[:3]  # Limit to 3 related files

```

That's a great catch\! Handling edge cases like empty files is essential for building resilient systems.

The bug is in how `file_record.content.split('\n')` is handled when `file_record.content` is an empty string (`""`). While splitting an empty string works (`"".split('\n')` returns `['']`), if the intent is to return an empty context for an empty file, we should check for it explicitly.

The main issue is more subtle, however: if `file_record.content` is `""`, `lines` becomes `['']`, and `len(lines)` is `1`. The code then proceeds to return `file_record.content`, which is `""`, and everything works fine **in this specific case**.

However, if an empty file were a **long** empty string (e.g., a file filled with only newlines, `"\n\n\n"`), or if we wanted to make it explicitly clear that a file with no content results in no context, adding an explicit check is still the cleanest solution.

Since the goal is to make it "more robust" and handle the empty content case gracefully, let's add the explicit check as requested.

Here is the fixed `get_file_context` function:

```python
from mock_data import Session, CodeFile
from typing import List, Dict


def get_file_context(session: Session, file_path: str, max_lines=50) -> str:
    """Get current file content as context, gracefully handling empty files."""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return ""
    
    # NEW CHECK: Handle files with explicitly empty content
    if not file_record.content:
        return ""
    
    lines = file_record.content.split('\n')
    
    # If the content was just a single empty string (i.e., the file was truly empty), 
    # the list will contain [''] and len will be 1. The check above handles the "" case.
    
    if len(lines) <= max_lines:
        return file_record.content
    
    # Return first part of file with truncation notice
    truncated = '\n'.join(lines[:max_lines])
    return f"{truncated}\n... [truncated after {max_lines} lines]"


def get_recent_changes(session: Session, file_path: str, limit=3) -> List[Dict]:
    """Get recent changes to the file"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return []
    
    # This is a stub for demonstration; in a real scenario, this would query commits
    # Here we return a mock list
    return [
        {
            'hash': 'abc12345',
            'message': 'Initial commit',
            'author': 'Alice',
            'date': '2024-06-01'
        },
        {
            'hash': 'def67890',
            'message': 'Refactor code',
            'author': 'Bob',
            'date': '2024-06-02'
        }
    ][:limit]


def find_related_files(session: Session, file_path: str) -> List[str]:
    """Find files that might be related based on imports"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return []
    
    # Simple heuristic: find files imported in this file
    related = []
    content = file_record.content
    
    # Python imports
    for line in content.split('\n'):
        if line.strip().startswith('from ') or line.strip().startswith('import '):
            # Extract module name and try to find corresponding file
            # This is a simplified approach
            if 'import' in line and '.' in line:
                module = line.split()[-1].replace('.', '/') + '.py'
                if session.query(CodeFile).filter_by(file_path=module).first():
                    related.append(module)
    
    return related[:3]  # Limit to 3 related files
```

-----

## 🛠️ Explanation of the Fix

The fix involves adding this concise line immediately after checking for the file record:

```python
    if not file_record.content:
        return ""
```

  * **Before the Fix:** If a `CodeFile` record was found but its `content` was an empty string (`""`), the code would proceed to `lines = file_record.content.split('\n')`. This is generally safe, but adding an explicit check for **empty content** ensures that:
    1.  The function immediately returns an empty context for an empty file, which is semantically correct.
    2.  The code clearly handles the edge case of an empty string, making the function more **readable and robust** against potential future changes where `split()` might behave differently or where the subsequent logic might make invalid assumptions about the content of `lines`.

## Handling Empty Files with Grace

You've just learned how to extract file content for AI code review! Now, let's improve our context generator by fixing a bug that appears when handling empty files.

Currently, our get_file_context function tries to split the content into lines without first checking whether that content exists. When a file is empty, this causes the function to process an empty string, which can lead to errors in real-world scenarios.

To fix this issue:

Run the tests to see the bug in action
Add a check that detects when file_record.content is empty
Return an empty string immediately when empty content is detected
This small improvement makes your context generator much more robust. In real code review systems, you'll often encounter empty files (such as newly created files or configuration templates), and your code needs to handle these cases smoothly without crashing.

```python
from typing import List, Dict
from mock_data import Session, CodeFile

def get_file_context(session: Session, file_path: str, max_lines=50) -> str:
    """Get current file content as context"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return ""
    
    # TODO: Add a check for empty content and return an empty string if the content is empty
    
    lines = file_record.content.split('\n')
    if len(lines) <= max_lines:
        return file_record.content
    
    # Return first part of file with truncation notice
    truncated = '\n'.join(lines[:max_lines])
    return f"{truncated}\n... [truncated after {max_lines} lines]"

def get_recent_changes(session: Session, file_path: str, limit=3) -> List[Dict]:
    """Get recent changes to the file"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return []
    
    # This is a stub for demonstration; in a real scenario, this would query commits
    # Here we return a mock list
    return [
        {
            'hash': 'abc12345',
            'message': 'Initial commit',
            'author': 'Alice',
            'date': '2024-06-01'
        },
        {
            'hash': 'def67890',
            'message': 'Refactor code',
            'author': 'Bob',
            'date': '2024-06-02'
        }
    ][:limit]

def find_related_files(session: Session, file_path: str) -> List[str]:
    """Find files that might be related based on imports"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return []
    
    # Simple heuristic: find files imported in this file
    related = []
    content = file_record.content
    
    # Python imports
    for line in content.split('\n'):
        if line.strip().startswith('from ') or line.strip().startswith('import '):
            # Extract module name and try to find corresponding file
            # This is a simplified approach
            if 'import' in line and '.' in line:
                module = line.split()[-1].replace('.', '/') + '.py'
                if session.query(CodeFile).filter_by(file_path=module).first():
                    related.append(module)
    
    return related[:3]  # Limit to 3 related files

```

## Handling Empty Files with Grace

You've just learned how to extract file content for AI code review! Now, let's improve our context generator by fixing a bug that appears when handling empty files.

Currently, our get_file_context function tries to split the content into lines without first checking whether that content exists. When a file is empty, this causes the function to process an empty string, which can lead to errors in real-world scenarios.

To fix this issue:

Run the tests to see the bug in action
Add a check that detects when file_record.content is empty
Return an empty string immediately when empty content is detected
This small improvement makes your context generator much more robust. In real code review systems, you'll often encounter empty files (such as newly created files or configuration templates), and your code needs to handle these cases smoothly without crashing.

```python
from typing import List, Dict
from mock_data import Session, CodeFile

def get_file_context(session: Session, file_path: str, max_lines=50) -> str:
    """Get current file content as context"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return ""
    
    # TODO: Add a check for empty content and return an empty string if the content is empty
    
    lines = file_record.content.split('\n')
    if len(lines) <= max_lines:
        return file_record.content
    
    # Return first part of file with truncation notice
    truncated = '\n'.join(lines[:max_lines])
    return f"{truncated}\n... [truncated after {max_lines} lines]"

def get_recent_changes(session: Session, file_path: str, limit=3) -> List[Dict]:
    """Get recent changes to the file"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return []
    
    # This is a stub for demonstration; in a real scenario, this would query commits
    # Here we return a mock list
    return [
        {
            'hash': 'abc12345',
            'message': 'Initial commit',
            'author': 'Alice',
            'date': '2024-06-01'
        },
        {
            'hash': 'def67890',
            'message': 'Refactor code',
            'author': 'Bob',
            'date': '2024-06-02'
        }
    ][:limit]

def find_related_files(session: Session, file_path: str) -> List[str]:
    """Find files that might be related based on imports"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return []
    
    # Simple heuristic: find files imported in this file
    related = []
    content = file_record.content
    
    # Python imports
    for line in content.split('\n'):
        if line.strip().startswith('from ') or line.strip().startswith('import '):
            # Extract module name and try to find corresponding file
            # This is a simplified approach
            if 'import' in line and '.' in line:
                module = line.split()[-1].replace('.', '/') + '.py'
                if session.query(CodeFile).filter_by(file_path=module).first():
                    related.append(module)
    
    return related[:3]  # Limit to 3 related files

```

## Truncating Long Files for Better Context

Now that you've learned about extracting file content for AI code review, let's improve our context generator by adding truncation functionality for long files. When reviewing code, it's often impractical to include very large files in their entirety.

The current implementation of get_file_context returns the complete file content regardless of size. In real-world scenarios, this could overwhelm the AI with too much information and waste tokens on less relevant parts of the file.

Your task is to modify the function to:

Split the file content into lines
Check whether the number of lines exceeds the max_lines parameter (default: 50)
If the file is short enough, return the full content
If the file is too long, return only the first max_lines lines, followed by a truncation notice
The truncation notice should follow this format: "... [truncated after X lines]", where X is the number of lines shown.

This improvement will make your context generator more practical for real-world code review scenarios, where files can be hundreds or thousands of lines long.

````python
from typing import List, Dict
from mock_data import Session, CodeFile

def get_file_context(session: Session, file_path: str, max_lines=50) -> str:
    """Get current file content as context"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return ""
    
    # TODO: Split the content into lines and check if it exceeds max_lines
    
    # TODO: If the file is short enough, return the full content
    
    # TODO: If the file is too long, truncate it and add a notice
    return file_record.content

def get_recent_changes(session: Session, file_path: str, limit=3) -> List[Dict]:
    """Get recent changes to the file"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return []
    
    # This is a stub for demonstration; in a real scenario, this would query commits
    # Here we return a mock list
    return [
        {
            'hash': 'abc12345',
            'message': 'Initial commit',
            'author': 'Alice',
            'date': '2024-06-01'
        },
        {
            'hash': 'def67890',
            'message': 'Refactor code',
            'author': 'Bob',
            'date': '2024-06-02'
        }
    ][:limit]

def find_related_files(session: Session, file_path: str) -> List[str]:
    """Find files that might be related based on imports"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return []
    
    # Simple heuristic: find files imported in this file
    related = []
    content = file_record.content
    
    # Python imports
    for line in content.split('\n'):
        if line.strip().startswith('from ') or line.strip().startswith('import '):
            # Extract module name and try to find corresponding file
            # This is a simplified approach
            if 'import' in line and '.' in line:
                module = line.split()[-1].replace('.', '/') + '.py'
                if session.query(CodeFile).filter_by(file_path=module).first():
                    related.append(module)
    
    return related[:3]  # Limit to 3 related files

````

Incorporating truncation is a smart way to manage costs and keep the AI's focus sharp\! You'll use Python's list slicing combined with the `max_lines` parameter to implement this effectively.

Here is the updated `get_file_context` function:

```python
from typing import List, Dict
from mock_data import Session, CodeFile

def get_file_context(session: Session, file_path: str, max_lines=50) -> str:
    """Get current file content as context, with truncation for long files."""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return ""
    
    # 1. Split the content into lines
    lines = file_record.content.split('\n')
    
    # 2. Check if the file is short enough
    if len(lines) <= max_lines:
        # Return the full content
        return file_record.content
    
    # 3. If the file is too long, truncate it
    # Take only the first max_lines
    truncated_lines = lines[:max_lines]
    truncated_content = '\n'.join(truncated_lines)
    
    # Add the truncation notice
    return f"{truncated_content}\n... [truncated after {max_lines} lines]"

def get_recent_changes(session: Session, file_path: str, limit=3) -> List[Dict]:
    """Get recent changes to the file"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return []
    
    # This is a stub for demonstration; in a real scenario, this would query commits
    # Here we return a mock list
    return [
        {
            'hash': 'abc12345',
            'message': 'Initial commit',
            'author': 'Alice',
            'date': '2024-06-01'
        },
        {
            'hash': 'def67890',
            'message': 'Refactor code',
            'author': 'Bob',
            'date': '2024-06-02'
        }
    ][:limit]

def find_related_files(session: Session, file_path: str) -> List[str]:
    """Find files that might be related based on imports"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return []
    
    # Simple heuristic: find files imported in this file
    related = []
    content = file_record.content
    
    # Python imports
    for line in content.split('\n'):
        if line.strip().startswith('from ') or line.strip().startswith('import '):
            # Extract module name and try to find corresponding file
            # This is a simplified approach
            if 'import' in line and '.' in line:
                module = line.split()[-1].replace('.', '/') + '.py'
                if session.query(CodeFile).filter_by(file_path=module).first():
                    related.append(module)
    
    return related[:3]  # Limit to 3 related files
```

The logic effectively **splits the file into a list of lines**, checks the list's length against `max_lines`, and uses Python's **slicing `[:max_lines]`** to grab only the relevant starting portion before joining it back into a single string with the required truncation notice.



## Tracking Code History for AI Review

Now that you've learned how to handle file content and empty files, let's move on to the next important piece of context for AI code review: tracking recent changes. Understanding how a file has evolved over time gives the AI valuable insight into the code's history and purpose.

In this exercise, you'll implement the get_recent_changes function, which returns a list of recent commits for a given file. The function should check whether the file exists and then return a structured list of commit information that the AI can use to understand the file's history.

Your task is to:

Complete the function by creating a list of dictionaries, where each dictionary represents a commit.
Include keys for hash, message, author, and date in each commit dictionary.
Make sure to respect the limit parameter to return only the requested number of changes.
This implementation will help the AI understand not just what the code looks like now, but how it has changed over time — a crucial part of providing meaningful code review feedback.


```python
from typing import List, Dict
from mock_data import Session, CodeFile

def get_file_context(session: Session, file_path: str, max_lines=50) -> str:
    """Get current file content as context"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return ""
    
    if not file_record.content:
        return ""
    
    lines = file_record.content.split('\n')
    if len(lines) <= max_lines:
        return file_record.content
    
    # Return first part of file with truncation notice
    truncated = '\n'.join(lines[:max_lines])
    return f"{truncated}\n... [truncated after {max_lines} lines]"

def get_recent_changes(session: Session, file_path: str, limit=3) -> List[Dict]:
    """Get recent changes to the file"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return []
    
    # TODO: Create a list of dictionaries representing recent commits
    # Each dictionary should have 'hash', 'message', 'author', and 'date' keys
    
    # TODO: Return the list of commits, respecting the limit parameter
    return []

def find_related_files(session: Session, file_path: str) -> List[str]:
    """Find files that might be related based on imports"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return []
    
    # Simple heuristic: find files imported in this file
    related = []
    content = file_record.content
    
    # Python imports
    for line in content.split('\n'):
        if line.strip().startswith('from ') or line.strip().startswith('import '):
            # Extract module name and try to find corresponding file
            # This is a simplified approach
            if 'import' in line and '.' in line:
                module = line.split()[-1].replace('.', '/') + '.py'
                if session.query(CodeFile).filter_by(file_path=module).first():
                    related.append(module)
    
    return related[:3]  # Limit to 3 related files

```

Great idea\! Giving the AI insight into a file's history is crucial for quality review. Since this is a mock environment, we'll implement the `get_recent_changes` function using **placeholder data** that respects the required structure and the `limit` parameter.

Here is the completed function:

```python
from typing import List, Dict
from mock_data import Session, CodeFile

def get_file_context(session: Session, file_path: str, max_lines=50) -> str:
    """Get current file content as context"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return ""
    
    if not file_record.content:
        return ""
    
    lines = file_record.content.split('\n')
    if len(lines) <= max_lines:
        return file_record.content
    
    # Return first part of file with truncation notice
    truncated = '\n'.join(lines[:max_lines])
    return f"{truncated}\n... [truncated after {max_lines} lines]"

def get_recent_changes(session: Session, file_path: str, limit=3) -> List[Dict]:
    """Get recent changes to the file"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return []
    
    # 1. Create a list of dictionaries representing recent commits (Mock Data)
    all_commits = [
        {
            'hash': 'f4c5a9b3',
            'message': 'Fix: Handle edge case where user name is empty.',
            'author': 'Alex Johnson',
            'date': '2025-09-20'
        },
        {
            'hash': 'c8e1d0f2',
            'message': 'Refactor: Simplify API response parsing logic.',
            'author': 'Bob Smith',
            'date': '2025-09-18'
        },
        {
            'hash': 'a2b7c4d1',
            'message': 'Feature: Initial implementation of settings page.',
            'author': 'Alice Green',
            'date': '2025-09-15'
        },
        {
            'hash': 'e9g3h1j0',
            'message': 'Chore: Update dependencies and cleanup old tests.',
            'author': 'Alex Johnson',
            'date': '2025-09-10'
        }
    ]
    
    # 2. Return the list of commits, respecting the limit parameter
    return all_commits[:limit]

def find_related_files(session: Session, file_path: str) -> List[str]:
    """Find files that might be related based on imports"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return []
    
    # Simple heuristic: find files imported in this file
    related = []
    content = file_record.content
    
    # Python imports
    for line in content.split('\n'):
        if line.strip().startswith('from ') or line.strip().startswith('import '):
            # Extract module name and try to find corresponding file
            # This is a simplified approach
            if 'import' in line and '.' in line:
                module = line.split()[-1].replace('.', '/') + '.py'
                if session.query(CodeFile).filter_by(file_path=module).first():
                    related.append(module)
    
    return related[:3]  # Limit to 3 related files
```

The function now provides a structured history, which is critical for helping the AI understand the **context** and **intent** behind the current state of the code\!

## Discovering Related Files for AI Context

Now that you've implemented file content extraction and tracked recent changes, let's complete our context generator by adding the ability to find related files! This is the final piece that helps the AI understand how different parts of the codebase are connected.

The find_related_files function discovers files that are imported by the current file, giving the AI reviewer a broader view of the code's dependencies. When reviewing a file, knowing which other files it relies on can provide crucial context.

Your task is to implement this function by:

Looping through each line of the file content
Identifying lines that start with from or import
Extracting module names that contain dots (like utils.helper)
Converting these module paths to file paths by replacing dots with slashes and adding .py
Checking if these files exist in the session
Adding existing files to the results list, limited to a maximum of 3
This feature will significantly improve your AI code reviewer's ability to understand the relationships between different files in your codebase, leading to more insightful and helpful reviews.

```python
from typing import List, Dict
from mock_data import Session, CodeFile

def get_file_context(session: Session, file_path: str, max_lines=50) -> str:
    """Get current file content as context"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return ""
    
    if not file_record.content:
        return ""
    
    lines = file_record.content.split('\n')
    if len(lines) <= max_lines:
        return file_record.content
    
    # Return first part of file with truncation notice
    truncated = '\n'.join(lines[:max_lines])
    return f"{truncated}\n... [truncated after {max_lines} lines]"

def get_recent_changes(session: Session, file_path: str, limit=3) -> List[Dict]:
    """Get recent changes to the file"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return []
    
    # This is a stub for demonstration; in a real scenario, this would query commits
    # Here we return a mock list
    return [
        {
            'hash': 'abc12345',
            'message': 'Initial commit',
            'author': 'Alice',
            'date': '2024-06-01'
        },
        {
            'hash': 'def67890',
            'message': 'Refactor code',
            'author': 'Bob',
            'date': '2024-06-02'
        },
        {
            'hash': 'ghi54321',
            'message': 'Fix bug in main function',
            'author': 'Charlie',
            'date': '2024-06-03'
        }
    ][:limit]

def find_related_files(session: Session, file_path: str) -> List[str]:
    """Find files that might be related based on imports"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return []
    
    # Simple heuristic: find files imported in this file
    related = []
    content = file_record.content
    
    # TODO: Loop through each line in the content
    
    # TODO: Check if the line starts with 'from' or 'import'
    
    # TODO: For lines with imports, extract module names that contain dots
    
    # TODO: Convert module names to file paths (replace dots with slashes, add .py)
    
    # TODO: Check if the file exists in the session and add it to related files
    
    # TODO: Return the list of related files, limited to 3 maximum
    return []

```

That's an excellent plan\! Implementing dependency tracking via imports is a significant step toward providing the AI with holistic context.

Here is the completed `find_related_files` function, using the logic you outlined to parse Python imports:

```python
from typing import List, Dict
from mock_data import Session, CodeFile

def get_file_context(session: Session, file_path: str, max_lines=50) -> str:
    """Get current file content as context"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return ""
    
    if not file_record.content:
        return ""
    
    lines = file_record.content.split('\n')
    if len(lines) <= max_lines:
        return file_record.content
    
    # Return first part of file with truncation notice
    truncated = '\n'.join(lines[:max_lines])
    return f"{truncated}\n... [truncated after {max_lines} lines]"

def get_recent_changes(session: Session, file_path: str, limit=3) -> List[Dict]:
    """Get recent changes to the file"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return []
    
    # This is a stub for demonstration; in a real scenario, this would query commits
    # Here we return a mock list
    return [
        {
            'hash': 'abc12345',
            'message': 'Initial commit',
            'author': 'Alice',
            'date': '2024-06-01'
        },
        {
            'hash': 'def67890',
            'message': 'Refactor code',
            'author': 'Bob',
            'date': '2024-06-02'
        },
        {
            'hash': 'ghi54321',
            'message': 'Fix bug in main function',
            'author': 'Charlie',
            'date': '2024-06-03'
        }
    ][:limit]

def find_related_files(session: Session, file_path: str) -> List[str]:
    """Find files that might be related based on imports"""
    file_record = session.query(CodeFile).filter_by(file_path=file_path).first()
    if not file_record:
        return []
    
    related = []
    content = file_record.content
    
    # Loop through each line in the content
    for line in content.split('\n'):
        line_stripped = line.strip()
        
        # Check if the line starts with 'from' or 'import'
        if line_stripped.startswith('from ') or line_stripped.startswith('import '):
            
            # Use a simplified heuristic to find dotted modules (e.g., utils.helper)
            if 'import' in line_stripped and '.' in line_stripped:
                try:
                    # Extract module name (assumes the module name is the last word)
                    # Example: 'from utils.helper import func' -> 'func' (not ideal, but works for simple cases)
                    # More robustly, let's grab the module path before 'import' or just the name.
                    
                    if line_stripped.startswith('from '):
                        # Extracts 'utils.helper' from 'from utils.helper import func'
                        module_path = line_stripped.split(' ')[1] 
                    elif line_stripped.startswith('import '):
                        # Extracts 'utils.helper' from 'import utils.helper'
                        module_path = line_stripped.split(' ')[1] 
                    else:
                        continue
                        
                    # Remove trailing commas or aliases if present (simple cleanup)
                    module_path = module_path.split(' as ')[0].rstrip(',')
                    
                    # Convert module names to file paths (replace dots with slashes, add .py)
                    # Example: utils.helper becomes utils/helper.py
                    file_name = module_path.replace('.', '/') + '.py'
                    
                    # Check if the file exists in the session
                    if session.query(CodeFile).filter_by(file_path=file_name).first():
                        if file_name not in related:
                            related.append(file_name)
                            
                except IndexError:
                    # Skip malformed lines
                    continue
    
    # Return the list of related files, limited to 3 maximum
    return related[:3] 
```

This implementation iterates through the lines, identifies import statements, converts Python module paths (`a.b.c`) into file paths (`a/b/c.py`), and validates their existence using the `session.query`, thus fulfilling all requirements\!